In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
import pandas as pd
import numpy as np
import nltk

df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML-2022/main/Lab-04/spam_detection.csv")
df.head()

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


In [4]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]           # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ
print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


In [5]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-11-17 08:02:30--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-17 08:02:30--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-17 08:02:30--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [6]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
## không cần hiểu đống này lắm đâu
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

# Hãy train lại model sao cho embedding layer có tính learnable Kèm theo đó là dùng bộ glove 6B 300d

In [8]:
vocab, matrix = load_word_embeddings("glove.6B.300d.txt")

In [9]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [10]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [11]:
# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, [[2,0],[0,0]], mode='constant', constant_values =0.0)
print(embedding_matrix)

# Khi đó, __PADDED_INDEX__ dùng dòng đầu tiên của  embedding_matrix
# __UNKNOWN_WORD__ dùng dòng thứ hai của embedding_matrix 

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.04656    0.21318   -0.0074364 ...  0.0090611 -0.20989    0.053913 ]
 ...
 [ 0.075713  -0.040502   0.18345   ...  0.21838    0.30967    0.43761  ]
 [ 0.81451   -0.36221    0.31186   ...  0.075486   0.28408   -0.17559  ]
 [ 0.429191  -0.296897   0.15011   ...  0.28975    0.32618   -0.0590532]]


In [12]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [13]:
## Chia data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state =0)

In [14]:

from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         120000600 
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 120,161,202
Trainable params: 160,602
Non-trainable params: 120,000,600
_________________________________________________________________


In [15]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="lstm_spam.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)
# Train
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
         epochs=5,
         callbacks=[mc])

model.load_weights("lstm_spam.h5")
_, val_acc = model.evaluate(X_valid, y_valid)
print("Accuracy on valid: ", val_acc)

Epoch 1/5
140/140 [==============================] - 183s 1s/step - loss: 0.1338 - accuracy: 0.9540 - val_loss: 0.0852 - val_accuracy: 0.9713
Epoch 2/5
140/140 [==============================] - 221s 2s/step - loss: 0.0599 - accuracy: 0.9812 - val_loss: 0.0625 - val_accuracy: 0.9785
Epoch 3/5
140/140 [==============================] - 229s 2s/step - loss: 0.0381 - accuracy: 0.9892 - val_loss: 0.0510 - val_accuracy: 0.9830
Epoch 4/5
140/140 [==============================] - 234s 2s/step - loss: 0.0263 - accuracy: 0.9924 - val_loss: 0.0474 - val_accuracy: 0.9848
Epoch 5/5
35/35 [==============================] - 4s 118ms/step - loss: 0.0474 - accuracy: 0.9848
Accuracy on valid:  0.9847533702850342


# Hãy viết một hàm

def model_predict(text):

return True/False

In [16]:
text_predict = "wanna ask something? just send me a mess"

In [23]:
def model_predict(text):
  text_predict = text.lower()
  text_predict = nltk.tokenize.word_tokenize(text_predict)
  cur_text_indices =[]
  for word in text_predict:
    if word in word_to_index:
        cur_text_indices.append(word_to_index[word])    ## map từ word sang index
    else:
        cur_text_indices.append(__UNKNOWN_WORD__) 
  cur_text_indices = np.array([cur_text_indices],dtype =np.int32)
  D = np.argmax(model.predict(cur_text_indices))
  if D==0:
    return False
  return True

In [27]:
df['Text'][:5]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: Text, dtype: object

In [29]:
for t in df['Text'][:5]:
  print(t)
  print(model_predict(t))


Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
1/1 [==============================] - 0s 107ms/step
False
Ok lar... Joking wif u oni...
1/1 [==============================] - 0s 34ms/step
False
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
1/1 [==============================] - 0s 57ms/step
True
U dun say so early hor... U c already then say...
1/1 [==============================] - 0s 34ms/step
False
Nah I don't think he goes to usf, he lives around here though
1/1 [==============================] - 0s 60ms/step
False


# Dùng model vừa train đoán xem các câu sau có phải spam không: 

*   "wanna ask something? just send me a mess"
*   "Urgent! You have won our competition's prize!! Please call us now."
*   "Call me to get a free holiday now"

In [30]:
texts_test = ["wanna ask something? just send me a mess",
              "Urgent! You have won our competition's prize!! Please call us now.",
              "Call me to get a free holiday now",]
for t in texts_test:
  print(t)
  print(model_predict(t))

wanna ask something? just send me a mess
1/1 [==============================] - 0s 22ms/step
False
Urgent! You have won our competition's prize!! Please call us now.
1/1 [==============================] - 0s 24ms/step
True
Call me to get a free holiday now
1/1 [==============================] - 0s 28ms/step
False
